<a href="https://colab.research.google.com/github/WeiChenTseng/Titanic-analysis/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9FDAY04_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#讀取資料
url = 'https://raw.githubusercontent.com/WeiChenTseng/Titanic-analysis/main/Titanic%20Data/train.csv'
ur2 = 'https://raw.githubusercontent.com/WeiChenTseng/Titanic-analysis/main/Titanic%20Data/test.csv'
df_train = pd.read_csv(url)
df_test = pd.read_csv(ur2)

In [3]:
# 將訓練集和測試集合併一起做資料前處理
df_data = df_train.append(df_test)
df_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


# 特徵工程

Name(姓名)

In [4]:
df_data['Title'] = df_data['Name'].str.split(', ',expand=True)[1]
df_data['Title'] = df_data['Title'].str.split('.', expand=True)[0]

In [5]:
df_data['Title'] = df_data.Title.replace( ['Don','Rev','Dr','Major','Lady','Sir','Col','Capt','Countess','Jonkheer','Dona'], 'Rare')
df_data['Title'] = df_data.Title.replace( ['Ms','Mlle'], 'Miss' )
df_data['Title'] = df_data.Title.replace( 'Mme', 'Mrs' )

In [6]:
df_data.drop('Name', axis=1, inplace=True)

船票號碼

In [7]:
df_data['Ticket_info'] =  df_data['Ticket'].apply(lambda x:x.replace('/','').replace('.','').strip().split(' ')[0] if not x.isdigit() else 'X')

In [8]:
df_data['Ticket_info'].unique()

array(['A5', 'PC', 'STONO2', 'X', 'PP', 'CA', 'SCParis', 'SCA4', 'A4',
       'SP', 'SOC', 'WC', 'SOTONOQ', 'WEP', 'STONO', 'C', 'SCPARIS',
       'SOP', 'Fa', 'LINE', 'FCC', 'SWPP', 'SCOW', 'PPP', 'SC', 'SCAH',
       'AS', 'SOPP', 'FC', 'SOTONO2', 'CASOTON', 'SCA3', 'STONOQ', 'AQ4',
       'A', 'LP', 'AQ3'], dtype=object)

In [9]:
df_data.drop('Ticket',axis=1,inplace=True)

填補缺漏值

In [10]:
#票價(Fare) : 填補整體票價的中位數
df_data['Fare'].fillna(df_data.Fare.median(), inplace=True)

In [11]:
#登船港口(Embarked) : 填補次數最多的港口 'S'
print(df_data['Embarked'].value_counts())
# 填補 Embarked 欄位的缺漏值
df_data['Embarked'].fillna('S',inplace=True)

S    914
C    270
Q    123
Name: Embarked, dtype: int64


In [12]:
#因此，我們將先計算每個稱謂的年齡平均值，再用此平均值來填補每個 Title 中所對應 Age 欄位的缺漏值。

# 計算每個 Title 的年齡平均值及中位數
Age_Mean = df_data[['Title','Age']].groupby(by=['Title']).mean()
Age_Median = df_data[['Title','Age']].groupby(by=['Title']).median()

Age_Mean.columns = ['Age Mean']
Age_Median.columns = ['Age Median']
Age_Mean.reset_index( inplace=True )
Age_Median.reset_index( inplace=True )

In [13]:
# 利用每個 Title 的年齡平均數，填補每個 Title 所對應 Age 的缺漏值
df_data.loc[(df_data.Age.isnull())&(df_data.Title=='Master'),'Age'] = Age_Mean.loc[Age_Mean.Title=='Master']['Age Mean'][0]
df_data.loc[(df_data.Age.isnull())&(df_data.Title=='Miss'),'Age'] = Age_Mean.loc[Age_Mean.Title=='Miss']['Age Mean'][1]
df_data.loc[(df_data.Age.isnull())&(df_data.Title=='Mr'),'Age'] = Age_Mean.loc[Age_Mean.Title=='Mr']['Age Mean'][2]
df_data.loc[(df_data.Age.isnull())&(df_data.Title=='Mrs'),'Age'] = Age_Mean.loc[Age_Mean.Title=='Mrs']['Age Mean'][3]
df_data.loc[(df_data.Age.isnull())&(df_data.Title=='Rare'),'Age'] = Age_Mean.loc[Age_Mean.Title=='Rare']['Age Mean'][4]

In [14]:
# 觀察 Cabin 中的種類
df_data['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [15]:
df_data['Cabin'] = df_data['Cabin'].apply( lambda x:str(x)[0] if not pd.isnull(x) else 'NoCabin' )
df_data['Cabin'].unique() #再次檢查是否還有缺漏值尚未填補到。

array(['NoCabin', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [16]:
# OneHot Encoding
OneHot_Embarked = pd.get_dummies( df_data.Embarked,prefix='Embarked') #在Columns 前面加上Embarked

df_data = pd.concat( [ df_data, OneHot_Embarked ], axis=1 )
df_data.drop( 'Embarked', axis=1, inplace=True )

In [17]:
# Label Encoding
Sex_mapping = { 'male':0, 'female':1 }
df_data[ 'Sex' ] = df_data.Sex.map( Sex_mapping )

In [18]:
df_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Title,Ticket_info,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,0,22.0,1,0,7.2500,NoCabin,Mr,A5,0,0,1
1,2,1.0,1,1,38.0,1,0,71.2833,C,Mrs,PC,1,0,0
2,3,1.0,3,1,26.0,0,0,7.9250,NoCabin,Miss,STONO2,0,0,1
3,4,1.0,1,1,35.0,1,0,53.1000,C,Mrs,X,0,0,1
4,5,0.0,3,0,35.0,0,0,8.0500,NoCabin,Mr,X,0,0,1


將文字轉換成數字

In [19]:
df_data['Title'] = df_data['Title'].astype('category').cat.codes

In [20]:
df_data['Ticket_info'] = df_data['Ticket_info'].astype('category').cat.codes
df_data['Cabin'] = df_data['Cabin'].astype('category').cat.codes

In [21]:
df_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Title,Ticket_info,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,0,22.000000,1,0,7.2500,7,2,2,0,0,1
1,2,1.0,1,1,38.000000,1,0,71.2833,2,3,14,1,0,0
2,3,1.0,3,1,26.000000,0,0,7.9250,7,1,31,0,0,1
3,4,1.0,1,1,35.000000,1,0,53.1000,2,3,36,0,0,1
4,5,0.0,3,0,35.000000,0,0,8.0500,7,2,36,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,0,32.252151,0,0,8.0500,7,2,2,0,0,1
414,1306,NaN,1,1,39.000000,0,0,108.9000,2,4,14,1,0,0
415,1307,NaN,3,0,38.500000,0,0,7.2500,7,2,28,0,0,1
416,1308,NaN,3,0,32.252151,0,0,8.0500,7,2,36,0,0,1


# 產生訓練集與測試集

In [22]:
# 產生訓練集和測試集
Train = df_data[pd.notnull(df_data.Survived)]
Test = df_data[pd.isnull(df_data.Survived)]

# 訓練集刪除 PassengerId 欄位；
# 測試集刪除 PassengerId 與 Survived 欄位
Train.drop(['PassengerId'],axis=1,inplace=True)
Test.drop(['PassengerId','Survived'],axis=1,inplace=True)

# 將測試集中的標籤欄位 Survived 單獨拆出
Y_Train = Train.Survived
X_Train = Train.drop(['Survived'], axis=1)

print(f'Shape of X_Train = {X_Train.shape} ')
print(f'Shape of Y_Train = {Y_Train.shape} ')
print(f'Shape of Test = {Test.shape}')

Shape of X_Train = (891, 12) 
Shape of Y_Train = (891,) 
Shape of Test = (418, 12)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# 建立模型

隨機森林

In [23]:
# 隨機森林參數：
# n_estimators: 樹的數量(default=10)。
# min_samples_leaf: 最終葉節點最少樣本數(default=1)；
#                   當樣本不大時，可不設定使用預設，若樣本數量非常大時，則推薦增加此參數值。
# min_samples_split:節點再劃分時所需的最小樣本數(default=2)；
#                   當樣本不大時，可不設定使用預設，若樣本數量非常大時，則推薦增加此參數值。
# oob_score: 是否採用袋外樣本(out-of-bag samples)來評估模型的準確度(default=False)。
# 決策樹(Decision Tree)
from sklearn.tree import DecisionTreeClassifier
# 隨機森林(Random Forest)
from sklearn.ensemble import RandomForestClassifier



RFC = RandomForestClassifier( n_estimators = 1000,
                              min_samples_split = 20,
                              min_samples_leaf = 1,
                              oob_score = True,
                              random_state = 1,
                              n_jobs = -1 )

In [24]:
# 訓練模型
RFC.fit( X_Train, Y_Train )
print( 'oob_score = {:.6f}'.format(RFC.oob_score_) )

oob_score = 0.828283


SVM 支援向量機(Support Vector Machine)

In [27]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [28]:
Y_pred = cross_val_predict(SVC(),X_Train,Y_Train,cv=10)
acc_svc = metrics.accuracy_score(Y_Train,Y_pred)
print(metrics.classification_report(Y_Train,Y_pred))
#混淆矩陣

              precision    recall  f1-score   support

         0.0       0.68      0.92      0.78       549
         1.0       0.71      0.30      0.43       342

    accuracy                           0.69       891
   macro avg       0.70      0.61      0.60       891
weighted avg       0.69      0.69      0.65       891



In [29]:
print('svc score:',acc_svc)

svc score: 0.6857463524130191


XGBoost

In [37]:
from sklearn.model_selection import train_test_split

In [41]:
X = Train[{'Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Title','Ticket_info','Embarked_C','Embarked_Q','Embarked_S'}]
y = Train['Survived']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=33)

In [45]:
from xgboost import XGBClassifier
xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)
print('The accuracy of eXtrene Gradient Boosting Classifier on testing set:',xgbc.score(X_test, y_test))

The accuracy of eXtrene Gradient Boosting Classifier on testing set: 0.8475336322869955


PyTorch

In [47]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [48]:
df_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Cabin', 'Title', 'Ticket_info', 'Embarked_C', 'Embarked_Q',
       'Embarked_S'],
      dtype='object')

In [49]:
# 將資料分成訓練集、測試集

train_data = df_data[:891] 
test_data = df_data[891:]

titanic_train_data_X = train_data.drop(['Survived'], axis=1)
titanic_train_data_Y = train_data['Survived']
titanic_test_data = test_data.drop(['Survived'], axis=1)

# 將資料轉換成Tensor 格式
train_data = torch.from_numpy(titanic_train_data_X.values).float()
train_label = torch.from_numpy(titanic_train_data_Y.values).float()
test_data = torch.from_numpy(titanic_test_data.values).float()

# 進行數據封裝
train_dataset = TensorDataset(train_data, train_label)
trainLoader = DataLoader(train_dataset, batch_size=4,
              shuffle=True,  num_workers=2)


In [50]:
import torch.nn as nn
import torch.nn.functional as F

# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(titanic_train_data_X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.fc(x)


net = Net()

In [51]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # 定义损失函数
optimizer = optim.Adam(net.parameters(), lr=0.001)  # 定义优化器

import time
start = time.time()
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data  # 获取数据
        optimizer.zero_grad()  # 清空梯度缓存

        outputs = net(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()  # 反向传播
        optimizer.step()  # 调整模型
        running_loss += loss.item()
        if i % 20 == 19:
            # 每 20 次迭代打印一次信息
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
print('Finish Traning! Total cost time: ', time.time()-start)

[1,    20] loss: 0.007
[1,    40] loss: 0.007
[1,    60] loss: 0.007
[1,    80] loss: 0.007
[1,   100] loss: 0.007
[1,   120] loss: 0.007
[1,   140] loss: 0.007
[1,   160] loss: 0.007
[1,   180] loss: 0.007
[1,   200] loss: 0.007
[1,   220] loss: 0.007
[2,    20] loss: 0.006
[2,    40] loss: 0.006
[2,    60] loss: 0.007
[2,    80] loss: 0.007
[2,   100] loss: 0.007
[2,   120] loss: 0.006
[2,   140] loss: 0.008
[2,   160] loss: 0.006
[2,   180] loss: 0.007
[2,   200] loss: 0.008
[2,   220] loss: 0.008
Finish Traning! Total cost time:  2.645226001739502


In [52]:
correct = 0
total = 0
# evaluating时不计算导数
with torch.no_grad():
    for data in trainLoader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' %
      (correct / total * 100))

Accuracy of the network on the 10000 test images: 61 %
